## Caso de Estudio 2 - Machine Learning
### Santiago Hoyos - Mariana Ramírez


El objetivo del caso de estudio es usar una red neuronal para clasificar datos en un data set que mide la calidad del vino. Se tienen dos clases de vino: blanco y tinto; donde la calidad se mide en una escala entre 3 y 9.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

In [5]:
import warnings
warnings.filterwarnings("ignore")

Se cargaron los datos de un archivo CSV que contenía información sobre propiedades del vino, incluyendo su calidad. Se observó que el conjunto de datos tenía 6497 registros y 13 columnas.

In [6]:
df=pd.read_csv('winequalityN.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed acidity         6487 non-null   float64
 2   volatile acidity      6489 non-null   float64
 3   citric acid           6494 non-null   float64
 4   residual sugar        6495 non-null   float64
 5   chlorides             6495 non-null   float64
 6   free sulfur dioxide   6497 non-null   float64
 7   total sulfur dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6488 non-null   float64
 10  sulphates             6493 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


In [7]:
df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


Se eliminaron las columnas de tipo "type" ya que se asumió que no eran relevantes para la tarea de predicción de calidad del vino. Luego, se codificaron las etiquetas de calidad utilizando one-hot encoding (dummies) para crear columnas binarias para cada categoría de calidad. Además, se eliminaron las filas con valores faltantes.

In [8]:
df = df.drop(['type'], axis=1)

In [9]:
df = pd.get_dummies(df,columns=['quality'])
df = df.dropna()

df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,0,0,0,1,0,0,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,0,0,0,1,0,0,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,0,0,0,1,0,0,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0,0,0,1,0,0,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0,0,0,1,0,0,0


# Train, Test, Validation

Se dividieron los datos en conjuntos de entrenamiento, validación y prueba. Se utilizó una proporción del 60% para entrenamiento, un 20% para validación y un 20% para prueba. Los conjuntos de datos se prepararon para alimentar la red neuronal, convirtiendo los datos en tensores y creando DataLoaders para proporcionarlos por lotes a la red.

In [10]:
y = df[['quality_3','quality_4','quality_5','quality_6','quality_7','quality_8','quality_9']]
X = df.drop(columns=['quality_3','quality_4','quality_5','quality_6','quality_7','quality_8','quality_9'])
X_train, X_test1, y_train, y_test1 = train_test_split(X, y, train_size = 0.6, test_size = 0.4)
X_val, X_test, y_val, y_test = train_test_split(X_test1, y_test1, train_size = 0.2, test_size = 0.2)
y.head()

,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9
0,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0
2,0,0,0,1,0,0,0
3,0,0,0,1,0,0,0
4,0,0,0,1,0,0,0


In [11]:
class MyDataset():

  def __init__(self, X, y):
    if isinstance(X, pd.DataFrame):
        X = X.values
    if isinstance(y, pd.DataFrame):
        y = y.values

    self.X = torch.tensor(X, dtype=torch.float32)
    self.y = torch.tensor(y, dtype=torch.float32)

  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]


In [12]:
train_sec=MyDataset(X_train, y_train)
test_sec=MyDataset(X_test, y_test)
val_sec=MyDataset(X_val, y_val)

In [13]:
train_data=DataLoader(train_sec, batch_size=3, shuffle=False, num_workers=0, collate_fn=None, pin_memory=False)

test_data=DataLoader(test_sec, batch_size=3, shuffle=False, num_workers=0, collate_fn=None, pin_memory=False)

val_data=DataLoader(val_sec, batch_size=3, shuffle=False, num_workers=0, collate_fn=None, pin_memory=False)

# Clase Net

Se definió la arquitectura de la red neuronal utilizando capas lineales y funciones de activación. La red constaba de tres capas lineales con dimensiones apropiadas para la tarea.

In [14]:
for i, (data, labels) in enumerate(test_data):
  print(data.shape, labels.shape)
  print(data,labels)
  break;

torch.Size([3, 11]) torch.Size([3, 7])
tensor([[8.0000e+00, 6.7000e-01, 3.0000e-01, 2.0000e+00, 6.0000e-02, 3.8000e+01,
         6.2000e+01, 9.9580e-01, 3.2600e+00, 5.6000e-01, 1.0200e+01],
        [8.9000e+00, 6.3500e-01, 3.7000e-01, 1.7000e+00, 2.6300e-01, 5.0000e+00,
         6.2000e+01, 9.9710e-01, 3.0000e+00, 1.0900e+00, 9.3000e+00],
        [6.6000e+00, 3.2000e-01, 4.7000e-01, 1.5600e+01, 6.3000e-02, 2.7000e+01,
         1.7300e+02, 9.9872e-01, 3.1800e+00, 5.6000e-01, 9.0000e+00]]) tensor([[0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.]])


In [15]:
class Net(nn.Module):
    def __init__(self):
    	# Define all the parameters of the net
        super(Net, self).__init__()
        self.fc1 = nn.Linear(11,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,7)

    def forward(self, x):
    	# Do the forward pass
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

In [16]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [17]:
model = Net()
optimizer= torch.optim.SGD(model.parameters(),lr= 0.001)
criterion= nn.MSELoss()

In [18]:
model.to(device)

Net(
  (fc1): Linear(in_features=11, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=7, bias=True)
)

In [19]:
def train_model(model, optimizer, loss_module, train_loader, valid_loader, num_epochs):
    valid_loss_min = np.inf

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        v_loss = 0.0

        for data, target in train_loader:
            data_inputs = data.to(device)
            data_labels = target.to(device)

            optimizer.zero_grad()
            preds = model(data_inputs)
            preds = preds.squeeze(dim=1)
            loss = loss_module(preds, data_labels)
            loss.backward()

            # Clip gradients to prevent explosion
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()
            train_loss += loss.item() * data.size(0)

        train_loss = train_loss / len(train_loader.dataset)

        model.eval()
        for data, target in valid_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            valid_loss = loss_module(output, target)
            v_loss += valid_loss.item() * data.size(0)

        valid_loss = v_loss / len(valid_loader.dataset)

        print(f'Epoch: {epoch}\tTraining Loss: {train_loss:.6f}\tValidation Loss: {valid_loss:.6f}')

        if valid_loss <= valid_loss_min:
            print(f'Validation loss decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}).  Saving model ...')
            torch.save(model.state_dict(), 'model_voice.pt')
            valid_loss_min = valid_loss


# Training

Se entrenó el modelo durante 6 epochs, imprimiendo los errores de entrenamiento y validación en cada época. El modelo se guardó como "model_voice.pt" si la pérdida de validación disminuyó.

In [20]:
train_model(model, optimizer,criterion,train_data,val_data,6)

Epoch: 0	Training Loss: 0.118547	Validation Loss: 0.098189
Validation loss decreased (inf --> 0.098189).  Saving model ...
Epoch: 1	Training Loss: 0.097992	Validation Loss: 0.096558
Validation loss decreased (0.098189 --> 0.096558).  Saving model ...
Epoch: 2	Training Loss: 0.096512	Validation Loss: 0.096177
Validation loss decreased (0.096558 --> 0.096177).  Saving model ...
Epoch: 3	Training Loss: 0.096012	Validation Loss: 0.096025
Validation loss decreased (0.096177 --> 0.096025).  Saving model ...
Epoch: 4	Training Loss: 0.095654	Validation Loss: 0.095732
Validation loss decreased (0.096025 --> 0.095732).  Saving model ...
Epoch: 5	Training Loss: 0.095083	Validation Loss: 0.095461
Validation loss decreased (0.095732 --> 0.095461).  Saving model ...


# Best Model

In [21]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)

fc1.weight tensor([[ 1.4755e-01,  7.5776e-02, -1.7629e-01, -1.4679e-01,  4.5320e-02,
          1.8755e-01,  2.4015e-01,  7.4579e-02, -2.0720e-01, -8.5652e-02,
          2.3368e-01],
        [ 2.3877e-01,  2.0091e-01, -1.0397e-01,  4.0179e-02,  2.2954e-01,
          1.9232e-01,  1.4267e-01, -5.8295e-02,  1.9562e-01,  1.9757e-01,
          6.9032e-02],
        [ 2.9526e-01, -8.7641e-03,  7.1962e-03, -2.2972e-01, -9.0119e-02,
          2.1418e-01,  1.6150e-01, -9.6305e-02,  2.7664e-01,  2.6885e-01,
         -2.4118e-01],
        [ 1.3460e-01,  1.2673e-01, -1.0317e-01,  3.9616e-02,  1.3410e-01,
          2.3311e-01, -1.6308e-01, -3.9831e-02,  2.4839e-01, -9.9720e-02,
         -1.5539e-01],
        [ 4.9143e-02,  8.6497e-02, -2.2731e-01,  1.2786e-01, -2.2026e-01,
          1.4486e-01, -9.5765e-02,  2.5872e-01,  1.3279e-01,  2.3579e-01,
          8.8361e-02],
        [-2.8280e-01,  4.3121e-02,  7.3996e-02,  2.9081e-01,  3.2807e-02,
         -2.6990e-02,  1.7648e-01,  1.6110e-02,  1.4272e-01,

In [22]:
test_loss=0.0

criterion= nn.MSELoss()
for data, target in test_data:
  data=data.to(device)
  target=target.to(device)
  output=model(data)
  loss= criterion(output,target)
  test_loss += loss.item()*data.size(0)
test_loss = test_loss/len(test_data.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

Test Loss: 0.092890



Se evaluó el desempeño del mejor modelo utilizando la métrica de precisión (accuracy). El resultado fue que el modelo obtuvo una precisión de aproximadamente 0.45 en el conjunto de prueba.

In [23]:
accuracy=0.0

for data, target in test_data:
    data=data.to(device)
    target=target.to(device)
    output=model(data).detach().numpy()
    target = np.argmax(target.detach().numpy(),axis=1)
    output = np.argmax(output,axis=1)
    a = accuracy_score(target,output)
    accuracy += a.item()*data.size(0)
accuracy = accuracy/len(test_data.dataset)
print('Accuracy: {:.6f}\n'.format(accuracy))

Accuracy: 0.474903



En general, el desempeño del modelo no fue muy alto, ya que la precisión es relativamente baja. Esto podría indicar que la arquitectura de la red o la configuración del entrenamiento pueden no ser adecuadas para este conjunto de datos específico. Sería recomendable realizar una exploración más exhaustiva de la arquitectura de la red, ajustar los hiperparámetros y posiblemente considerar técnicas de preprocesamiento de datos adicionales para mejorar el rendimiento del modelo.

# Incrementar la métrica

Para mejorar la métrica de precisión del modelo de predicción de calidad del vino, vamos a explorar varias estrategias, incluyendo:

* Selección de Atributos: Utilizaremos un árbol de decisión para la selección de atributos, utilizaremos un árbol de decisión para obtener la importancia de cada atributo en la predicción de calidad del vino. Luego, seleccionaremos los atributos más importantes y entrenaremos el modelo con ellos.

In [24]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier

import torch
import torch.nn as nn
import torch.optim as optim

y = df[['quality_3','quality_4','quality_5','quality_6','quality_7','quality_8','quality_9']]
X = df.drop(columns=['quality_3','quality_4','quality_5','quality_6','quality_7','quality_8','quality_9'])

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Selección de características
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
selector = SelectFromModel(tree, prefit=True)
X_train_sel = selector.transform(X_train)
X_test_sel = selector.transform(X_test)

# Definición de modelo
class Net(nn.Module):
    def __init__(self, n_features):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_features, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 7)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net(X_train_sel.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

X_train_sel = torch.from_numpy(X_train_sel).float()
X_test_sel = torch.from_numpy(X_test_sel).float()
# Entrenamiento
num_epochs = 100
y_train = torch.from_numpy(y_train.values).float()
for epoch in range(num_epochs):
    preds = model(X_train_sel)
    loss = criterion(preds, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Evaluación
with torch.no_grad():
    preds = model(X_test_sel)
    preds = preds.detach().numpy()
    y_true = np.argmax(y_test.values, axis=1)
    y_pred = np.argmax(preds, axis=1)
    accuracy = accuracy_score(y_true, y_pred)
    print("Accuracy:", accuracy)

Accuracy: 0.4176334106728538


La selección de características en este caso particular provocó una disminución en el performance del modelo en lugar de una mejora. Es posible que todas las características originales sean relevantes para la predicción y al eliminar algunas se perdió información útil.
El método de selección de características (SelectFromModel con árbol de decisión) quizás no fue el más adecuado para este problema. Se podrían probar otros métodos.

# Estandarización de los Datos

In [25]:
from sklearn.preprocessing import StandardScaler

# Estandarización de los datos
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_val_std = scaler.transform(X_val)
X_test_std = scaler.transform(X_test)

# Creación de dataloaders con datos estandarizados
train_data_std = MyDataset(X_train_std, y_train)
val_data_std = MyDataset(X_val_std, y_val)
test_data_std = MyDataset(X_test_std, y_test)

train_loader_std = DataLoader(train_data_std, batch_size=3, shuffle=True)
val_loader_std = DataLoader(val_data_std, batch_size=3, shuffle=False)
test_loader_std = DataLoader(test_data_std, batch_size=3, shuffle=False)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(11, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 7)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

# Creación y entrenamiento del modelo
model_std = Net()
optimizer_std = torch.optim.SGD(model_std.parameters(), lr=0.001)
criterion_std = nn.MSELoss()
model_std.to(device)

train_model(model_std, optimizer_std, criterion_std, train_loader_std, val_loader_std, num_epochs=6)

# Evaluación del modelo con datos estandarizados
test_loss_std = 0.0

for data, target in test_loader_std:
    data = data.to(device)
    target = target.to(device)
    output = model_std(data)
    loss = criterion_std(output, target)
    test_loss_std += loss.item() * data.size(0)

test_loss_std = test_loss_std / len(test_loader_std.dataset)

# Cálculo de la precisión
accuracy_std = 0.0

for data, target in test_loader_std:
    data = data.to(device)
    target = target.to(device)
    output = model_std(data).detach().numpy()
    target = np.argmax(target.detach().numpy(), axis=1)
    output = np.argmax(output, axis=1)
    a = accuracy_score(target, output)
    accuracy_std += a.item() * data.size(0)

accuracy_std = accuracy_std / len(test_loader_std.dataset)
print('Accuracy: {:.6f}\n'.format(accuracy_std))


Epoch: 0	Training Loss: 0.128025	Validation Loss: 0.103029
Validation loss decreased (inf --> 0.103029).  Saving model ...
Epoch: 1	Training Loss: 0.097450	Validation Loss: 0.092575
Validation loss decreased (0.103029 --> 0.092575).  Saving model ...
Epoch: 2	Training Loss: 0.091615	Validation Loss: 0.089379
Validation loss decreased (0.092575 --> 0.089379).  Saving model ...
Epoch: 3	Training Loss: 0.089383	Validation Loss: 0.087809
Validation loss decreased (0.089379 --> 0.087809).  Saving model ...
Epoch: 4	Training Loss: 0.088104	Validation Loss: 0.086790
Validation loss decreased (0.087809 --> 0.086790).  Saving model ...
Epoch: 5	Training Loss: 0.087247	Validation Loss: 0.086106
Validation loss decreased (0.086790 --> 0.086106).  Saving model ...
Accuracy: 0.510441



La estandarización de los datos puede mejorar los resultados de un modelo de redes neuronales. La estandarización transforma los datos para que tengan una media cercana a cero y una desviación estándar cercana a uno. Esto hace que los datos estén en una escala similar y reduce la variabilidad en las características, lo que puede hacer que el entrenamiento sea más estable.

Al tener datos estandarizados, el optimizador puede converger más rápido durante el entrenamiento. Las actualizaciones de los pesos de la red pueden realizarse de manera más eficiente, lo que puede acelerar la convergencia hacia una solución óptima.

En las redes neuronales, especialmente en funciones de activación como la función sigmoide o tangente hiperbólica, los valores muy grandes o muy pequeños pueden llevar a problemas de desbordamiento (overflow o underflow) numérico. La estandarización ayuda a mantener los valores en un rango apropiado, evitando estos problemas.

Cuando se utiliza la estandarización, los coeficientes de las características en el modelo entrenado tienen interpretaciones más directas. Pueden indicar la importancia relativa de cada característica en la predicción.

# Ajuste de hiperparámetros

In [26]:
import itertools

class Nethiper(nn.Module):
    def __init__(self, hidden_units, activation_function):
        super(Nethiper, self).__init__()
        self.fc1 = nn.Linear(11, hidden_units)
        self.fc2 = nn.Linear(hidden_units, 7)
        self.activation_function = activation_function

    def forward(self, x):
        x = self.activation_function(self.fc1(x))
        x = self.fc2(x)
        return x

# Definir listas de valores a probar para hiperparámetros
learning_rates = [0.001, 0.01]
hidden_units_list = [32, 64]
num_epochs_list = [5, 10]
activation_functions = [nn.ReLU(), nn.Tanh()]

# Variable para realizar un seguimiento del mejor resultado
best_result = None

for lr, hidden_units, num_epochs, activation_function in itertools.product(learning_rates, hidden_units_list, num_epochs_list, activation_functions):
    # Creación y entrenamiento del modelo con hiperparámetros específicos
    model = Nethiper(hidden_units, activation_function)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    model.to(device)

    train_model(model, optimizer, criterion, train_loader_std, val_loader_std, num_epochs)

    # Evaluación del modelo con datos de prueba
    test_loss_std = 0.0
    accuracy_std = 0.0

    for data, target in test_loader_std:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        loss = criterion(output, target)
        test_loss_std += loss.item() * data.size(0)

        output = output.detach().numpy()
        target = np.argmax(target.detach().numpy(), axis=1)
        output = np.argmax(output, axis=1)
        a = accuracy_score(target, output)
        accuracy_std += a.item() * data.size(0)

    test_loss_std = test_loss_std / len(test_loader_std.dataset)
    accuracy_std = accuracy_std / len(test_loader_std.dataset)

    result = {
        'lr': lr,
        'hidden_units': hidden_units,
        'num_epochs': num_epochs,
        'activation_function': str(activation_function),
        'test_loss': test_loss_std,
        'accuracy': accuracy_std
    }

    # Actualiza el mejor resultado si es necesario
    if best_result is None or result['accuracy'] > best_result['accuracy']:
        best_result = result

# Imprimir solo el mejor resultado
if best_result is not None:
    print("Mejor resultado:")
    print(f"Learning Rate: {best_result['lr']}, Hidden Units: {best_result['hidden_units']}, Epochs: {best_result['num_epochs']}, Activation: {best_result['activation_function']}")
    print(f"Test Loss: {best_result['test_loss']:.6f}, Accuracy: {best_result['accuracy']:.6f}")


Epoch: 0	Training Loss: 0.154874	Validation Loss: 0.112652
Validation loss decreased (inf --> 0.112652).  Saving model ...
Epoch: 1	Training Loss: 0.106769	Validation Loss: 0.102412
Validation loss decreased (0.112652 --> 0.102412).  Saving model ...
Epoch: 2	Training Loss: 0.100305	Validation Loss: 0.098695
Validation loss decreased (0.102412 --> 0.098695).  Saving model ...
Epoch: 3	Training Loss: 0.097149	Validation Loss: 0.096252
Validation loss decreased (0.098695 --> 0.096252).  Saving model ...
Epoch: 4	Training Loss: 0.095074	Validation Loss: 0.094634
Validation loss decreased (0.096252 --> 0.094634).  Saving model ...
Epoch: 0	Training Loss: 0.164347	Validation Loss: 0.123633
Validation loss decreased (inf --> 0.123633).  Saving model ...
Epoch: 1	Training Loss: 0.114000	Validation Loss: 0.102374
Validation loss decreased (0.123633 --> 0.102374).  Saving model ...
Epoch: 2	Training Loss: 0.100687	Validation Loss: 0.095482
Validation loss decreased (0.102374 --> 0.095482).  Sav

La mejora se logra al implementar un seguimiento en tiempo real de los resultados durante la búsqueda de hiperparámetros. Esto permite que el código compare constantemente la precisión de cada conjunto de hiperparámetros con el mejor resultado encontrado hasta el momento.

La clave está en actualizar automáticamente el "mejor resultado" si el resultado actual es más preciso que el anterior. Al final de la búsqueda, el código solo imprime el conjunto de hiperparámetros con la mayor precisión, facilitando la identificación de la configuración óptima sin necesidad de examinar todos los resultados manualmente. Esto hace que el proceso sea más eficiente y efectivo.

# Dos nuevas arquitecturas:

In [27]:
# Creación de dataloaders con datos estandarizados
train_data = MyDataset(X_train, y_train)
val_data = MyDataset(X_val, y_val)
test_data = MyDataset(X_test, y_test)

train_loader_std = DataLoader(train_data, batch_size=3, shuffle=True)
val_loader_std = DataLoader(val_data, batch_size=3, shuffle=False)
test_loader_std = DataLoader(test_data, batch_size=3, shuffle=False)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(11, 64)
        self.fc2 = nn.Linear(64, 7)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        return x

# Creación y entrenamiento del modelo
model_std = Net()
optimizer_std = torch.optim.SGD(model_std.parameters(), lr=0.001)
criterion_std = nn.MSELoss()
model_std.to(device)

train_model(model_std, optimizer_std, criterion_std, train_loader_std, val_loader_std, num_epochs=6)

# Evaluación del modelo con datos estandarizados
test_loss_std = 0.0

for data, target in test_loader_std:
    data = data.to(device)
    target = target.to(device)
    output = model_std(data)
    loss = criterion_std(output, target)
    test_loss_std += loss.item() * data.size(0)

test_loss_std = test_loss_std / len(test_loader_std.dataset)

# Cálculo de la precisión
accuracy_std = 0.0

for data, target in test_loader_std:
    data = data.to(device)
    target = target.to(device)
    output = model_std(data).detach().numpy()
    target = np.argmax(target.detach().numpy(), axis=1)
    output = np.argmax(output, axis=1)
    a = accuracy_score(target, output)
    accuracy_std += a.item() * data.size(0)

accuracy_std = accuracy_std / len(test_loader_std.dataset)
print('Accuracy: {:.6f}\n'.format(accuracy_std))

Epoch: 0	Training Loss: 0.129600	Validation Loss: 0.099984
Validation loss decreased (inf --> 0.099984).  Saving model ...
Epoch: 1	Training Loss: 0.099010	Validation Loss: 0.097993
Validation loss decreased (0.099984 --> 0.097993).  Saving model ...
Epoch: 2	Training Loss: 0.097452	Validation Loss: 0.097301
Validation loss decreased (0.097993 --> 0.097301).  Saving model ...
Epoch: 3	Training Loss: 0.096836	Validation Loss: 0.096704
Validation loss decreased (0.097301 --> 0.096704).  Saving model ...
Epoch: 4	Training Loss: 0.096268	Validation Loss: 0.096510
Validation loss decreased (0.096704 --> 0.096510).  Saving model ...
Epoch: 5	Training Loss: 0.095994	Validation Loss: 0.097687
Accuracy: 0.418407



In [28]:
# Creación de dataloaders con datos estandarizados
train_data = MyDataset(X_train, y_train)
val_data = MyDataset(X_val, y_val)
test_data = MyDataset(X_test, y_test)

train_loader_std = DataLoader(train_data, batch_size=3, shuffle=True)
val_loader_std = DataLoader(val_data, batch_size=3, shuffle=False)
test_loader_std = DataLoader(test_data, batch_size=3, shuffle=False)

class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.fc1 = nn.Linear(11, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 7)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model2 = Net2()
optimizer_std = torch.optim.SGD(model_std.parameters(), lr=0.001)
criterion_std = nn.MSELoss()
model_std.to(device)

train_model(model_std, optimizer_std, criterion_std, train_loader_std, val_loader_std, num_epochs=6)

# Evaluación del modelo con datos estandarizados
test_loss_std = 0.0

for data, target in test_loader_std:
    data = data.to(device)
    target = target.to(device)
    output = model_std(data)
    loss = criterion_std(output, target)
    test_loss_std += loss.item() * data.size(0)

test_loss_std = test_loss_std / len(test_loader_std.dataset)

# Cálculo de la precisión
accuracy_std = 0.0

for data, target in test_loader_std:
    data = data.to(device)
    target = target.to(device)
    output = model_std(data).detach().numpy()
    target = np.argmax(target.detach().numpy(), axis=1)
    output = np.argmax(output, axis=1)
    a = accuracy_score(target, output)
    accuracy_std += a.item() * data.size(0)

accuracy_std = accuracy_std / len(test_loader_std.dataset)
print('Accuracy: {:.6f}\n'.format(accuracy_std))

Epoch: 0	Training Loss: 0.095879	Validation Loss: 0.096881
Validation loss decreased (inf --> 0.096881).  Saving model ...
Epoch: 1	Training Loss: 0.095681	Validation Loss: 0.096452
Validation loss decreased (0.096881 --> 0.096452).  Saving model ...
Epoch: 2	Training Loss: 0.095563	Validation Loss: 0.096255
Validation loss decreased (0.096452 --> 0.096255).  Saving model ...
Epoch: 3	Training Loss: 0.095409	Validation Loss: 0.097216
Epoch: 4	Training Loss: 0.095389	Validation Loss: 0.096598
Epoch: 5	Training Loss: 0.095289	Validation Loss: 0.096278
Accuracy: 0.427688



Vemos que no necesariamente mejor. Por lo anterior, es posible afirmar que las mejores técnicas para incrementar la métrica serían las estandarización de datos y el ajuste de hiperparámetros, siendo esta última la de mejor resultado a pesar de su coste computacional.

In [29]:
import torch
import torch.nn as nn
import numpy as np

# Definir la clase Nethiper con ReLU como función de activación
class Nethiper(nn.Module):
    def __init__(self, hidden_units):
        super(Nethiper, self).__init__()
        self.fc1 = nn.Linear(11, hidden_units)
        self.fc2 = nn.Linear(hidden_units, 7)
        self.activation_function = nn.ReLU()

    def forward(self, x):
        x = self.activation_function(self.fc1(x))
        x = self.fc2(x)
        return x

# Cargar el modelo entrenado
best_model = Nethiper(best_result['hidden_units'])
best_model.load_state_dict(torch.load('model_voice.pt'))
best_model.eval()

# Seleccionar un registro aleatorio del conjunto de datos
random_data = np.random.rand(11)

# Convertir el registro aleatorio en un tensor de PyTorch
sample_data_tensor = torch.tensor(random_data, dtype=torch.float32)
sample_data_tensor = sample_data_tensor.unsqueeze(0)  # Añadir una dimensión adicional para el lote

# Realizar una predicción en el registro seleccionado
output = best_model(sample_data_tensor)

# Mapear la clase predicha a las etiquetas de calidad
predicted_class = torch.argmax(output).item() + 3  # Suma 3 para mapear a las etiquetas de calidad

print("Predicción para el Registro Aleatorio:")
print(f"Clase Predicha: {predicted_class}")


Predicción para el Registro Aleatorio:
Clase Predicha: 9
